In [ ]:
import requests as rq
import bs4 as bs4
import time
import json
import pandas as pd
import numpy as np
import math
import time

# Minerando dados Editoras

In [ ]:
# Url onde estão todas as editoras

url_editoras = 'https://www.skoob.com.br/editoras/'

In [ ]:
# Fazendo request do link e transformando em um objeto do bs4

response = rq.get(url_editoras)
pagina_html = bs4.BeautifulSoup(response.text)

In [ ]:
# Procurando tags

tags = pagina_html.findAll(id="resultadoBusca-")
editoras = tags[0].findAll('strong')

lista_editoras = []
lista_links = []

for x in editoras:
    if x.find('a') != None:
        editora = x.get_text()
        lista_editoras.append(editora)
        
        link = x.find('a').get('href')
        lista_links.append(link)
        
livros_editora = []
numero_livros = tags[0].findAll('span')

for livros in range(1, len(numero_livros), 8):
    n_livros = numero_livros[livros].get_text().split()[0]
    n_livros = n_livros.replace('.', '')
    livros_editora.append(n_livros)

In [ ]:
# Criando dataframe que armazenará os dados

df = pd.DataFrame()
df['editora'] = lista_editoras
df['link'] = lista_links
df['qtde_livros'] = livros_editora

In [ ]:
# Função para ser aplicada em cada linha do dataframe na coluna código
# Parâmetro x: string -> link da editora
# Irá retornar o código da editora

def codigo(x):
    return x.split('/')[-1].split('-')[0]

codigos = df['link'].apply(codigo)
df['codigo'] = codigos

In [ ]:
# Dataframe com as editoras

df.to_csv('./editoras_link.csv', index=False)

# Minerando livros e link de cada editora


Os livros de cada editora estão no link : "https://www.skoob.com.br/editora/livros/codigoDeCadaEditora"  
É necessário o código de cada editora para entrar na página dos livros. Os códigos foram salvos no editoras_link.csv

In [ ]:
lista_livros = []
lista_livros_link = []

for index, row in df.iterrows():
    print('--------------------------')
    print(df['editora'][index])
    
    
    url_base = 'https://www.skoob.com.br/editora/livros/{}/mpage:{}'
    
    # Número de páginas com livros
    # Cada página do site mostra 40 livros
    n_page = math.ceil(int(df['qtde_livros'][index])/40) + 1
    
    for x in range(1, n_page):
        print(x)
        url_livros = url_base.format(df['codigo'][index], x)
        
        response = rq.get(url_livros)
        pagina_html = bs4.BeautifulSoup(response.text)
        
        lista_div = pagina_html.findAll("div", {"class": "box_livro"})
        
        for x in lista_div:
            titulo = x.find('h3').get_text()
            link = x.find('a').get('href')
            
            lista_livros.append(titulo)
            lista_livros_link.append(link)
        
        time.sleep(1)


In [ ]:
# Criando dataframe

df = pd.DataFrame()
df['livros'] = lista_livros
df['link'] = lista_livros_link

In [ ]:
# Dataframe com as editoras

df.to_csv('./livros_link.csv', index=False)

# Livros

In [ ]:
df = pd.read_csv('./livros_link.csv')

In [ ]:
info_livros = []

for index, row in df.iterrows():
    
    livro = {}
    
    # De 10 em 10 request espera 1 segundo
    if index % 10 == 0:
        time.sleep(1)
    
    
    try:
        # Request para a paǵina do livro
        url_base = 'https://www.skoob.com.br{}'.format(df['link'][index])        
        response = rq.get(url_base)
        pagina_html = bs4.BeautifulSoup(response.text)
        
        # Titulo
        div_lateral = pagina_html.find("div", {"id": "pg-livro-menu-principal"})
        livro['titulo'] = div_lateral.find('strong').get_text()

        #print(f'{index} - {titulo}')
        #print(url_base)
        
        # Autor
        livro['autor'] = div_lateral.findAll('a')[1].get_text()
        
        # ISBN 
        div_menor = div_lateral.find('div', {"class": "sidebar-desc"})
        isbns = div_menor.findAll('span')
        try:
            livro['ISBN_13'] = str(isbns[0]).split('>')[1].split('<')[0]
            livro['ISBN_10'] = str(isbns[1]).split('>')[1].split('<')[0]
        except:
            livro['ISBN_13'] = ''
            livro['ISBN_10'] = ''
            
        
        # Texto com dados
        texto = div_menor.get_text()
        
        try:    
            livro['ano'] = texto.split('Ano: ')[1].split(' ')[0]
        except:
            livro['ano'] = ''
        
        try:
            livro['paginas'] = texto.split('Páginas: ')[1].split(' ')[0]
        except:
            livro['paginas'] = ''
            
        try:
            livro['idioma'] = texto.split('Idioma: ')[1].split(' ')[0]
        except:
            livro['idioma'] = ''
        
        try:
            livro['editora'] = texto.split('Editora: ')[1].split(' ')[0]
        except:
            livro['editora'] = ''
            

        div_superior = pagina_html.find('div', {'id': 'livro-perfil-status'})
        
        try:
             livro['rating'] = div_superior.find('span', {'class': 'rating'}).get_text()
        except:
             livro['rating'] = ''
        
        try:
            livro['avaliacao'] = div_superior.find('div', {'id': 'pg-livro-box-rating-avaliadores-numero'}).get_text().split(' ')[0]
        except:
            livro['avaliacao'] = ''


        info = div_superior.findAll('a', {'class': 'text_blue'})
        
        try:
            livro['resenha'] = info[0].get_text()
        except:
            livro['resenha'] = ''
            
        try:
            livro['abandonos']  = info[1].get_text()
        except:
            livro['abandonos'] = ''
        
        try:
            livro['relendo']  = info[2].get_text()
        except:
            livro['relendo'] = ''
            
        try:   
            livro['querem_ler'] = info[3].get_text()
        except:
            livro['querem_ler'] = ''
        
        try:
            livro['lendo'] = info[4].get_text()
        except:
            livro['lendo'] = ''
            
        try:
            livro['leram'] = info[5].get_text()
        except:
            livro['leram'] = ''


        info = pagina_html.find('p', {'itemprop': 'description'})
        
        try: 
            descricao = info.get_text().replace('\n', '')
            descricao = descricao.split('.')
            genero = descricao[-1]
            descricao = ''.join(descricao[:-1])
            livro['descricao'] = descricao
            livro['genero'] = genero
        except:
            livro['descricao'] = ''
            livro['genero'] = ''
            
        
        try: 
            info = pagina_html.find('span', {'class': 'pg-livro-icone-male-label'})
            livro['male'] = int(info.get_text().replace('%', ''))

            info = pagina_html.find('span', {'class': 'pg-livro-icone-female-label'})
            livro['female'] = int(info.get_text().replace('%', ''))
        
        except: 
            livro['male'] = 0
            livro['female'] = 0
            
        info_livros.append(livro)
        
    except Exception as e:
        print(e)
        continue

In [ ]:
# Algumas páginas possuem um formato diferente para mostrar o nome do autor
# As instâncias que estão com erro possuem autor como : \nR$ \n

for index, row in dd.iterrows():
    
    # Verificando se autor está errado
    if dd['autor'][index] == '\nR$ \n':
        
        livro = dd['titulo'][index]
        link = df[df['livros'] == livro].reset_index()
        
        try:
            url_base = 'https://www.skoob.com.br{}'.format(link['link'][0])
            response = rq.get(url_base)
            pagina_html = bs4.BeautifulSoup(response.text)
        except:
            continue
        
        try:
            autor = pagina_html.find("i", {"class": "sidebar-subtitulo"}).get_text()
        except:
            autor = ''
            print(url_base)
            
        dd.at[index, 'autor'] = autor
        print(autor)


In [ ]:
# Salvando dataset atualizado

dd.to_csv('./dados.csv', index=False)

In [ ]:
# Alguns livros possuem outro formato no nome do autor
# +/- 25 livros, vou apenas dropar esses registros

linhas_drop = df[df['autor'] == '\nR$ \n'].index
df.drop(index= linhas_drop, inplace=True)
df.to_csv('dados_ok_2.csv', index=False)

# Organizando dados

In [ ]:
# Verificar linhas vazias em cada coluna

for x in df.columns:

    nan_rows = df[df[x].isnull()]
    print(f"Número de linhas vazias na coluna: {x} = {nan_rows.shape[0]}")

In [ ]:
# Carregando datasets

livros_link = pd.read_csv('./livros_link.csv')
df = pd.read_csv('dados.csv')

In [ ]:
# Correção dos dados faltantes da coluna de autores

for index, row in df[df['autor'].isnull()].iterrows():
    
    livro = df['titulo'][index]
    link = livros_link[livros_link['livros'] == livro].reset_index()
    url_base = 'https://www.skoob.com.br{}'.format(link['link'][0])
    
    response = rq.get(url_base)
    pagina_html = bs4.BeautifulSoup(response.text)
    
    try:
        autor = pagina_html.find("h3", {"class": "secao-subtitulo"}).get_text()
    except:
        autor = ''
        
    df.at[index, 'autor'] = autor
    print(autor)

# Salvando dataset com dados novos
df.to_csv('./dados.csv', index=False)

In [ ]:
# Corrigindo ISBN
# Transformando dado em str

df['ISBN_10'] = df['ISBN_10'].astype('str')
df['ISBN_13'] = df['ISBN_13'].astype('str')

for index, row in df.iterrows():
    
    try:
        # Removendo .
        df.at[index, 'ISBN_13'] = df['ISBN_13'][index].split('.')[0]
    except:
        pass

In [ ]:
# Transformando ano em um inteiro

def to_int(x):
    try:
        return int(x)
    except:
        return(0)

df['ano'] = df['ano'].apply(to_int)

In [ ]:
# Corrigindo tipo das variáveis

df['paginas'] = df['paginas'].apply(to_int)
df['idioma'] = df['idioma'].astype('str')
df['avaliacao'] = df['avaliacao'].astype('int')
df['resenha'] = df['resenha'].astype('int')
df['abandonos'] = df['abandonos'].astype('int')
df['relendo'] = df['relendo'].astype('int')
df['querem_ler'] = df['querem_ler'].astype('int')
df['lendo'] = df['lendo'].astype('int')
df['leram'] = df['leram'].astype('int')
df['descricao'] = df['descricao'].astype('str')
df['genero'] = df['genero'].astype('str')
df['editora'] = df['editora'].astype('str')

In [ ]:
# Salvando dataset final

df.to_csv('./dados.csv', index=False)